In [23]:
# Date: January 19, 2017
# Author: Oliver Plunket
#
# Description: a script to be used on csv files containing artist rosters and organization verifications for burden-api
# galleries.  The script will output two csvs ready to imported into burden by way of the ImportArtistRostersService
# and the ImportOrganizationVerificationsService.  This script requires the following headers:
#
#      Burden ID, Name, Domain, Type of Organization, Artists are..., Artists
#

In [24]:
import pandas as pd

# import csv exported from G Sheets
df = pd.read_csv('/Users/oliver/Downloads/outsourcers.csv', encoding='utf-8')

# clean headers
df = df.rename(columns={'Burden ID': 'organization_id',
                        'Name': 'name',
                        'Type of Organization': 'organization_type',
                        'Artists are...': 'artist_type'})

# only data rows
df = df.loc[~(df.organization_id.isnull() | df.organization_id.str.contains('[a-zA-Z]', regex=True))]

***Organization Verifications***

In [25]:
# columns
cols = ['organization_id', 'organization_type']
org_vers = df[cols].fillna('')
org_vers['notes'] = ''
org_vers['score'] = ''

# turn answers into scores
org_vers.score.loc[org_vers.organization_type == 'Gallery'] = 2
org_vers.score.loc[org_vers.organization_type != 'Gallery'] = 1
org_vers.notes.loc[~org_vers.organization_type.str.contains('(Gallery|Other)', regex=True)] = org_vers.organization_type.str.lower()

org_vers = org_vers.drop('organization_type', axis=1)
org_vers = org_vers.sort_values(by='organization_id').reset_index(drop=True)

/Users/oliver/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [26]:
org_vers.head()

,organization_id,notes,score
0,10029,,1
1,10079,,2
2,10135,,2
3,10239,,1
4,10358,,1


In [5]:
org_vers.to_csv('', encoding='utf-8', index=False)

IOError: [Errno 2] No such file or directory: ''

***Artist Rosters***

In [ ]:
df = df.fillna('')

headers = list(df.columns.values)
artist_headers = headers[headers.index('Notes')+1:]
rosters_headers = headers[:headers.index('artist_type')+1]

rosters = pd.DataFrame()

# takes a minute
for i in df.index.values:
    row = df.ix[i]
    for x in artist_headers:
        if row[x] != '':
            artist = row[x]
            single_artist_row = row[rosters_headers]
            single_artist_row['artist'] = artist
            rosters = rosters.append(single_artist_row, ignore_index=True)

In [ ]:
# cleanup
for x in df.columns.values:
    df[x] = df[x].str.strip()

rosters.artist_type.loc[rosters.artist_type == 'Unknown'] = ''
rosters.artist = rosters.artist.str.title()
rosters.artist_type = rosters.artist_type.str.lower()

cols = ['organization_id', 'artist', 'artist_type']
rosters = rosters[cols]
# current project isn't collecting notes
rosters['notes'] = ''

for header in rosters.columns.values:
    rosters[header] = rosters[header].str.strip()

rosters = rosters.sort_values(by='organization_id').reset_index(drop=True)

In [ ]:
if set(rosters.artist_type.value_counts().index.values) != set(['', 'exhibited', 'represented']):
    print 'Invalid artist_type. Only exhibited and represented are valid types.'

In [ ]:
rosters.head()

In [22]:
rosters.to_csv('test_roster_csv.csv', encoding='utf-8', index=False)